In [13]:
from Levenshtein import ratio

def partial_ratio(str1, str2):
    if len(str1) < len(str2):
        str1, str2 = str2, str1  # Ensure str1 is the longer string

    best_score = 0

    # Generate all substrings of the shorter string
    for start in range(len(str1)):
        for end in range(start + 1, len(str1) + 1):
            substring = str1[start:end]
            # Calculate similarity score with the longer string
            score = ratio(substring, str1)
            # Calculate the score as a percentage
            best_score = max(best_score, score)

    return best_score*100

# Example usage
string1 = "i shouldn't have said that"
string2 = "Probably shouldn't have said that."

score = partial_ratio(string1, string2)
print(f"Partial Ratio Score: {score}%")  # Output: 100.00%


Partial Ratio Score: 100.0%
